In [137]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import datetime
import hashlib
import random
from sklearn.linear_model import MultiTaskLassoCV
%matplotlib inline
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook

In [9]:
cassava = pd.read_csv('CassavaImagelinks.csv')
cassava.shape

(135607, 22)

## Cleaning the data frame

In [11]:
#checking for duplicate rows
cassava.duplicated() 

0         False
1         False
2         False
3         False
4         False
          ...  
135602    False
135603    False
135604    False
135605    False
135606    False
Length: 135607, dtype: bool

In [12]:
#removing duplicated rows
cassava.drop_duplicates(inplace=True)
cassava                                   #result: There were no duplicated rows

,_URI,_CREATOR_URI_USER,_CREATION_DATE,_LAST_UPDATE_URI_USER,_LAST_UPDATE_DATE,_MODEL_VERSION,_UI_VERSION,_IS_COMPLETE,_SUBMISSION_DATE,_MARKED_AS_COMPLETE_DATE,...,META_INSTANCE_ID,CASSAVA_DATA_START_TIME,CASSAVA_DATA_CASSAVA_COMMENT,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT,CASSAVA_DATA_PHONE_ID,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC,IMAGE_LINK
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450000,NaN,2019-09-23 00:37:16.450000,NaN,NaN,1,2019-09-23 00:37:16.443000,2019-09-23 00:37:16.443000,...,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458000,NaN,2019-09-23 13:08:04.458000,NaN,NaN,1,2019-09-23 13:08:04.451000,2019-09-23 13:08:04.451000,...,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196000,NaN,2019-09-23 13:08:21.196000,NaN,NaN,1,2019-09-23 13:08:21.189000,2019-09-23 13:08:21.189000,...,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384000,NaN,2019-09-23 13:08:26.384000,NaN,NaN,1,2019-09-23 13:08:26.377000,2019-09-23 13:08:26.377000,...,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028000,NaN,2019-09-23 13:08:37.028000,NaN,NaN,1,2019-09-23 13:08:37.012000,2019-09-23 13:08:37.012000,...,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135602,uuid:87d9de62-de01-4470-937d-57035d14e9f3,anonymousUser,2020-06-30 04:09:25.013000,NaN,2020-06-30 04:09:25.013000,NaN,NaN,0,2020-06-30 04:09:24.999000,NaN,...,uuid:87d9de62-de01-4470-937d-57035d14e9f3,2020-06-26 09:54:05.501000,NaN,1118.500000,3.582461e+14,33.649752,2.043023,2020-06-26 00:00:00,4.940,http://3.19.236.194:8080/ODKAggregate/view/bin...
135603,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,anonymousUser,2020-06-30 04:09:25.807000,NaN,2020-06-30 04:09:25.807000,NaN,NaN,0,2020-06-30 04:09:25.801000,NaN,...,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,2020-06-26 09:56:58.989000,NaN,1118.700000,3.582461e+14,33.649744,2.043025,2020-06-26 00:00:00,4.150,http://3.19.236.194:8080/ODKAggregate/view/bin...
135604,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,anonymousUser,2020-06-30 04:09:26.606000,NaN,2020-06-30 04:09:26.606000,NaN,NaN,0,2020-06-30 04:09:26.592000,NaN,...,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,2020-06-26 09:57:23.987000,NaN,1118.300000,3.582461e+14,33.649717,2.042974,2020-06-26 00:00:00,4.240,http://3.19.236.194:8080/ODKAggregate/view/bin...
135605,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,anonymousUser,2020-06-30 04:09:27.389000,NaN,2020-06-30 04:09:27.389000,NaN,NaN,0,2020-06-30 04:09:27.383000,NaN,...,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,2020-06-26 09:57:50.049000,NaN,1118.500000,3.582461e+14,33.649718,2.042978,2020-06-26 00:00:00,4.100,http://3.19.236.194:8080/ODKAggregate/view/bin...


In [13]:
#treat spaces as null values
#parse the dates
cassava = pd.read_csv('CassavaImagelinks.csv', na_values=['',' '], parse_dates=["_CREATION_DATE","_LAST_UPDATE_DATE","_MARKED_AS_COMPLETE_DATE","_SUBMISSION_DATE"])
print(cassava)

                                             _URI _CREATOR_URI_USER  \
0       uuid:793b08fd-4d24-42a0-835c-fede2e71ec14     anonymousUser   
1       uuid:e235fa16-536d-4e2b-a463-95d32407f852     anonymousUser   
2       uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee     anonymousUser   
3       uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96     anonymousUser   
4       uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48     anonymousUser   
...                                           ...               ...   
135602  uuid:87d9de62-de01-4470-937d-57035d14e9f3     anonymousUser   
135603  uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e     anonymousUser   
135604  uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c     anonymousUser   
135605  uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c     anonymousUser   
135606  uuid:ea6f2587-377d-4342-95bb-1446613d388b     anonymousUser   

                _CREATION_DATE _LAST_UPDATE_URI_USER       _LAST_UPDATE_DATE  \
0      2019-09-23 00:37:16.450                   NaN 2019-09-23 00:

In [14]:
#checking for information about the data
cassava.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135607 entries, 0 to 135606
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   _URI                                135607 non-null  object        
 1   _CREATOR_URI_USER                   135607 non-null  object        
 2   _CREATION_DATE                      135607 non-null  datetime64[ns]
 3   _LAST_UPDATE_URI_USER               1896 non-null    object        
 4   _LAST_UPDATE_DATE                   135607 non-null  datetime64[ns]
 5   _MODEL_VERSION                      0 non-null       float64       
 6   _UI_VERSION                         0 non-null       float64       
 7   _IS_COMPLETE                        135607 non-null  int64         
 8   _SUBMISSION_DATE                    135607 non-null  datetime64[ns]
 9   _MARKED_AS_COMPLETE_DATE            99907 non-null   datetime64[ns]
 10  CASSAVA_

In [15]:
cassava.columns

Index(['_URI', '_CREATOR_URI_USER', '_CREATION_DATE', '_LAST_UPDATE_URI_USER',
       '_LAST_UPDATE_DATE', '_MODEL_VERSION', '_UI_VERSION', '_IS_COMPLETE',
       '_SUBMISSION_DATE', '_MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_DATA_CASSAVA_IMAGE_CLASS', 'META_INSTANCE_ID',
       'CASSAVA_DATA_START_TIME', 'CASSAVA_DATA_CASSAVA_COMMENT',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT', 'CASSAVA_DATA_PHONE_ID',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT', 'CASSAVA_DATA_SUBMISSION_TIME',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC', 'IMAGE_LINK'],
      dtype='object')

In [16]:
#replacing NaN values in the chosen columns
#Columns with object datatype were replaced with an empty string
#Columns with a float datatype were replaced with a zero

cassava['_LAST_UPDATE_URI_USER'].fillna("", inplace=True)
cassava['_MODEL_VERSION'].fillna(0, inplace=True)
cassava['_UI_VERSION'].fillna(0, inplace=True)
cassava['CASSAVA_DATA_CASSAVA_COMMENT'].fillna("", inplace=True)
cassava

,_URI,_CREATOR_URI_USER,_CREATION_DATE,_LAST_UPDATE_URI_USER,_LAST_UPDATE_DATE,_MODEL_VERSION,_UI_VERSION,_IS_COMPLETE,_SUBMISSION_DATE,_MARKED_AS_COMPLETE_DATE,...,META_INSTANCE_ID,CASSAVA_DATA_START_TIME,CASSAVA_DATA_CASSAVA_COMMENT,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT,CASSAVA_DATA_PHONE_ID,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC,IMAGE_LINK
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450,,2019-09-23 00:37:16.450,0.0,0.0,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,...,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458,,2019-09-23 13:08:04.458,0.0,0.0,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,...,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196,,2019-09-23 13:08:21.196,0.0,0.0,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,...,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384,,2019-09-23 13:08:26.384,0.0,0.0,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,...,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028,,2019-09-23 13:08:37.028,0.0,0.0,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,...,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135602,uuid:87d9de62-de01-4470-937d-57035d14e9f3,anonymousUser,2020-06-30 04:09:25.013,,2020-06-30 04:09:25.013,0.0,0.0,0,2020-06-30 04:09:24.999,NaT,...,uuid:87d9de62-de01-4470-937d-57035d14e9f3,2020-06-26 09:54:05.501000,,1118.500000,3.582461e+14,33.649752,2.043023,2020-06-26 00:00:00,4.940,http://3.19.236.194:8080/ODKAggregate/view/bin...
135603,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,anonymousUser,2020-06-30 04:09:25.807,,2020-06-30 04:09:25.807,0.0,0.0,0,2020-06-30 04:09:25.801,NaT,...,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,2020-06-26 09:56:58.989000,,1118.700000,3.582461e+14,33.649744,2.043025,2020-06-26 00:00:00,4.150,http://3.19.236.194:8080/ODKAggregate/view/bin...
135604,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,anonymousUser,2020-06-30 04:09:26.606,,2020-06-30 04:09:26.606,0.0,0.0,0,2020-06-30 04:09:26.592,NaT,...,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,2020-06-26 09:57:23.987000,,1118.300000,3.582461e+14,33.649717,2.042974,2020-06-26 00:00:00,4.240,http://3.19.236.194:8080/ODKAggregate/view/bin...
135605,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,anonymousUser,2020-06-30 04:09:27.389,,2020-06-30 04:09:27.389,0.0,0.0,0,2020-06-30 04:09:27.383,NaT,...,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,2020-06-26 09:57:50.049000,,1118.500000,3.582461e+14,33.649718,2.042978,2020-06-26 00:00:00,4.100,http://3.19.236.194:8080/ODKAggregate/view/bin...


In [17]:
cassava.head()

,_URI,_CREATOR_URI_USER,_CREATION_DATE,_LAST_UPDATE_URI_USER,_LAST_UPDATE_DATE,_MODEL_VERSION,_UI_VERSION,_IS_COMPLETE,_SUBMISSION_DATE,_MARKED_AS_COMPLETE_DATE,...,META_INSTANCE_ID,CASSAVA_DATA_START_TIME,CASSAVA_DATA_CASSAVA_COMMENT,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT,CASSAVA_DATA_PHONE_ID,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG,CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC,IMAGE_LINK
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450,,2019-09-23 00:37:16.450,0.0,0.0,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,...,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458,,2019-09-23 13:08:04.458,0.0,0.0,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,...,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196,,2019-09-23 13:08:21.196,0.0,0.0,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,...,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384,,2019-09-23 13:08:26.384,0.0,0.0,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,...,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028,,2019-09-23 13:08:37.028,0.0,0.0,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,...,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...


In [18]:
cassava.columns

Index(['_URI', '_CREATOR_URI_USER', '_CREATION_DATE', '_LAST_UPDATE_URI_USER',
       '_LAST_UPDATE_DATE', '_MODEL_VERSION', '_UI_VERSION', '_IS_COMPLETE',
       '_SUBMISSION_DATE', '_MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_DATA_CASSAVA_IMAGE_CLASS', 'META_INSTANCE_ID',
       'CASSAVA_DATA_START_TIME', 'CASSAVA_DATA_CASSAVA_COMMENT',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT', 'CASSAVA_DATA_PHONE_ID',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT', 'CASSAVA_DATA_SUBMISSION_TIME',
       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC', 'IMAGE_LINK'],
      dtype='object')

In [19]:
#rename the columns so as to easily understand the data
cassava.rename(columns={'_URI':'ID','_CREATOR_URI_USER':'CREATOR_USER_ID','_CREATION_DATE':'CREATION_DATE','_LAST_UPDATE_URI_USER':'LAST_UPDATE_USER_ID',
                        '_LAST_UPDATE_DATE':'LAST_UPDATE_DATE','_MODEL_VERSION':'MODEL_VERSION',
                       '_UI_VERSION':'UI_VERSION','_IS_COMPLETE':'IS_COMPLETE','_SUBMISSION_DATE':'SUBMISSION_DATE',
                       '_MARKED_AS_COMPLETE_DATE':'MARKED_AS_COMPLETE_DATE','CASSAVA_DATA_CASSAVA_IMAGE_CLASS':'CASSAVA_IMAGE_CLASS','CASSAVA_DATA_CASSAVA_COMMENT':'CASSAVA_DATA_COMMENT',
                       'CASSAVA_DATA_PHONE_ID':'CASSAVA_DEVICE_ID','CASSAVA_DATA_CASSAVA_IMAGE_GPS_ALT':'CASSAVA_GPS_ALT','CASSAVA_DATA_CASSAVA_IMAGE_GPS_LNG':'CASSAVA_GPS_LNG',
                       'CASSAVA_DATA_CASSAVA_IMAGE_GPS_LAT':'CASSAVA_GPS_LAT','CASSAVA_DATA_CASSAVA_IMAGE_GPS_ACC':'CASSAVA_GPS_ACC'}, inplace=True)
cassava.head()


,ID,CREATOR_USER_ID,CREATION_DATE,LAST_UPDATE_USER_ID,LAST_UPDATE_DATE,MODEL_VERSION,UI_VERSION,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,...,META_INSTANCE_ID,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_GPS_ACC,IMAGE_LINK
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450,,2019-09-23 00:37:16.450,0.0,0.0,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,...,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458,,2019-09-23 13:08:04.458,0.0,0.0,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,...,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196,,2019-09-23 13:08:21.196,0.0,0.0,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,...,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384,,2019-09-23 13:08:26.384,0.0,0.0,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,...,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028,,2019-09-23 13:08:37.028,0.0,0.0,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,...,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...


In [20]:
#to view new column names
cassava.columns

Index(['ID', 'CREATOR_USER_ID', 'CREATION_DATE', 'LAST_UPDATE_USER_ID',
       'LAST_UPDATE_DATE', 'MODEL_VERSION', 'UI_VERSION', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'META_INSTANCE_ID', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'CASSAVA_DATA_SUBMISSION_TIME',
       'CASSAVA_GPS_ACC', 'IMAGE_LINK'],
      dtype='object')

## Anonymizing sensitive user data (device id column) using the hashing method.

In [138]:
cassava['CASSAVA_DEVICE_ID'] = [hashlib.sha1(str.encode(str(i))).hexdigest() for i in cassava['CASSAVA_DEVICE_ID']]
display(cassava.head())

,ID,CREATOR_USER_ID,CREATION_DATE,LAST_UPDATE_USER_ID,LAST_UPDATE_DATE,MODEL_VERSION,UI_VERSION,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,...,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_GPS_ACC,IMAGE_LINK,month,month_year
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450,,2019-09-23 00:37:16.450,0.0,0.0,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,...,Necrosis,0.000000,11e623a37e87cf7995c466723ec99688d55cae8c,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...,9,2019-09
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458,,2019-09-23 13:08:04.458,0.0,0.0,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,...,This plant was planted when it was sick,0.000000,3e5467bcbcb8f58c3652330b411d2c8f977d4e00,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...,9,2019-09
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196,,2019-09-23 13:08:21.196,0.0,0.0,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,...,Yellow ish leaves,1137.299927,9b5ae6256ef75e52de9b5966f5c540ab49480488,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...,9,2019-09
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384,,2019-09-23 13:08:26.384,0.0,0.0,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,...,Healthy,1134.400000,5a4bc2394c367b568736478d6408edb020e6b445,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...,9,2019-09
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028,,2019-09-23 13:08:37.028,0.0,0.0,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,...,Clean cassava tuber,1147.500000,5a4bc2394c367b568736478d6408edb020e6b445,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...,9,2019-09


In [21]:
# drop the meta_instance_id column because it has same meaning as the ID/URI column
cassava.drop(['META_INSTANCE_ID'], axis=1)
cassava

,ID,CREATOR_USER_ID,CREATION_DATE,LAST_UPDATE_USER_ID,LAST_UPDATE_DATE,MODEL_VERSION,UI_VERSION,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,...,META_INSTANCE_ID,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,CASSAVA_DATA_SUBMISSION_TIME,CASSAVA_GPS_ACC,IMAGE_LINK
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,anonymousUser,2019-09-23 00:37:16.450,,2019-09-23 00:37:16.450,0.0,0.0,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,...,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09-23 00:00:00,2500.000,http://3.19.236.194:8080/ODKAggregate/view/bin...
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,anonymousUser,2019-09-23 13:08:04.458,,2019-09-23 13:08:04.458,0.0,0.0,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,...,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09-23 00:00:00,92.900,http://3.19.236.194:8080/ODKAggregate/view/bin...
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,anonymousUser,2019-09-23 13:08:21.196,,2019-09-23 13:08:21.196,0.0,0.0,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,...,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09-23 00:00:00,7.300,http://3.19.236.194:8080/ODKAggregate/view/bin...
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,anonymousUser,2019-09-23 13:08:26.384,,2019-09-23 13:08:26.384,0.0,0.0,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,...,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09-23 00:00:00,4.992,http://3.19.236.194:8080/ODKAggregate/view/bin...
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,anonymousUser,2019-09-23 13:08:37.028,,2019-09-23 13:08:37.028,0.0,0.0,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,...,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09-23 00:00:00,4.320,http://3.19.236.194:8080/ODKAggregate/view/bin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135602,uuid:87d9de62-de01-4470-937d-57035d14e9f3,anonymousUser,2020-06-30 04:09:25.013,,2020-06-30 04:09:25.013,0.0,0.0,0,2020-06-30 04:09:24.999,NaT,...,uuid:87d9de62-de01-4470-937d-57035d14e9f3,2020-06-26 09:54:05.501000,,1118.500000,3.582461e+14,33.649752,2.043023,2020-06-26 00:00:00,4.940,http://3.19.236.194:8080/ODKAggregate/view/bin...
135603,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,anonymousUser,2020-06-30 04:09:25.807,,2020-06-30 04:09:25.807,0.0,0.0,0,2020-06-30 04:09:25.801,NaT,...,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,2020-06-26 09:56:58.989000,,1118.700000,3.582461e+14,33.649744,2.043025,2020-06-26 00:00:00,4.150,http://3.19.236.194:8080/ODKAggregate/view/bin...
135604,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,anonymousUser,2020-06-30 04:09:26.606,,2020-06-30 04:09:26.606,0.0,0.0,0,2020-06-30 04:09:26.592,NaT,...,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,2020-06-26 09:57:23.987000,,1118.300000,3.582461e+14,33.649717,2.042974,2020-06-26 00:00:00,4.240,http://3.19.236.194:8080/ODKAggregate/view/bin...
135605,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,anonymousUser,2020-06-30 04:09:27.389,,2020-06-30 04:09:27.389,0.0,0.0,0,2020-06-30 04:09:27.383,NaT,...,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,2020-06-26 09:57:50.049000,,1118.500000,3.582461e+14,33.649718,2.042978,2020-06-26 00:00:00,4.100,http://3.19.236.194:8080/ODKAggregate/view/bin...


In [22]:
cassava.columns

Index(['ID', 'CREATOR_USER_ID', 'CREATION_DATE', 'LAST_UPDATE_USER_ID',
       'LAST_UPDATE_DATE', 'MODEL_VERSION', 'UI_VERSION', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'META_INSTANCE_ID', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'CASSAVA_DATA_SUBMISSION_TIME',
       'CASSAVA_GPS_ACC', 'IMAGE_LINK'],
      dtype='object')

## Creating a smaller dataframe that contains only required/relevant columns

In [24]:
#Locate the columns I need
new_cassava = cassava.iloc[:,[0,2,4,7,8,9,10,11,13,14,15,16,17,18]]
new_cassava.head(20)

,ID,CREATION_DATE,LAST_UPDATE_DATE,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,CASSAVA_DATA_END_TIME,CASSAVA_IMAGE_CLASS,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:37:16.450,2019-09-23 00:37:16.450,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,2019-09-23 00:36:31.097000,other,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 13:08:04.458,2019-09-23 13:08:04.458,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,2019-09-23 12:21:17.279000,cmd,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 13:08:21.196,2019-09-23 13:08:21.196,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,2019-09-23 12:38:01.552000,healthy,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 13:08:26.384,2019-09-23 13:08:26.384,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,2019-09-23 12:22:28.894000,healthy,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 13:08:37.028,2019-09-23 13:08:37.028,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,2019-09-23 12:23:32.876000,healthy,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361
5,uuid:b4e3caf0-ae1c-40aa-b4bc-600ae42bf293,2019-09-23 13:08:43.511,2019-09-23 13:08:43.511,1,2019-09-23 13:08:43.505,2019-09-23 13:08:43.505,2019-09-23 12:40:44.173000,cmd,2019-09-23 12:38:17.720000,Yellowish spots for,1137.299927,3.582461e+14,32.625310,0.521428
6,uuid:009e0994-c733-4686-9250-85e45732ee7c,2019-09-23 13:09:00.177,2019-09-23 13:09:00.177,1,2019-09-23 13:09:00.163,2019-09-23 13:09:00.163,2019-09-23 12:25:13.604000,cbb,2019-09-23 12:23:48.310000,CBB affected plant,1128.700000,3.582461e+14,32.625422,0.521151
7,uuid:fdac7c9b-14da-46ee-aa5b-0d20f7f0a789,2019-09-23 13:09:10.231,2019-09-23 13:09:10.231,1,2019-09-23 13:09:10.224,2019-09-23 13:09:10.224,2019-09-23 12:42:10.800000,cbb,2019-09-23 12:41:00.594000,Disease,1137.299927,3.582461e+14,32.625304,0.521480
8,uuid:54da5973-f3bb-4ca1-affe-3818093522c9,2019-09-23 13:09:48.821,2019-09-23 13:09:48.821,1,2019-09-23 13:09:48.806,2019-09-23 13:09:48.806,2019-09-23 12:27:35.180000,other,2019-09-23 12:25:29.019000,Sooty symptomatic plant,1138.200000,3.582461e+14,32.625339,0.521506
9,uuid:5b927f01-0742-4502-aac6-73429b2dfec4,2019-09-23 13:10:59.152,2019-09-23 13:10:59.152,1,2019-09-23 13:10:59.143,2019-09-23 13:10:59.143,2019-09-23 12:29:33.186000,cbsd,2019-09-23 12:27:56.247000,Candy stem lesion,1138.200000,3.582461e+14,32.625292,0.521515


In [25]:
new_cassava.columns

Index(['ID', 'CREATION_DATE', 'LAST_UPDATE_DATE', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT'],
      dtype='object')

In [26]:
new_cassava.head(15).iloc[:,[10,7]]


,CASSAVA_GPS_ALT,CASSAVA_IMAGE_CLASS
0,0.000000,other
1,0.000000,cmd
2,1137.299927,healthy
3,1134.400000,healthy
4,1147.500000,healthy
5,1137.299927,cmd
6,1128.700000,cbb
7,1137.299927,cbb
8,1138.200000,other
9,1138.200000,cbsd


## Grouping the data by different columns


In [125]:
dict(list(new_cassava.groupby('IS_COMPLETE')))[0]


,ID,CREATION_DATE,LAST_UPDATE_DATE,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,CASSAVA_DATA_END_TIME,CASSAVA_IMAGE_CLASS,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,mnth_yr,month,diff,time_diff
43,uuid:18c549b9-dfeb-4483-b842-95bf678ab7e6,2019-09-25 07:54:02.157,2019-10-03 09:16:21.213,0,2019-09-25 07:54:02.146,NaT,2019-09-25 07:13:31.250000,other,2019-09-25 07:12:46.700000,Rate mole,1039.6,3.582581e+14,33.126314,0.982232,2019-09,9,0,-1 days +23:59:15.450000
44,uuid:a0b5b3c6-abdf-4a57-8da3-19d48ba6970d,2019-09-25 07:54:07.537,2019-10-03 09:16:22.104,0,2019-09-25 07:54:07.502,NaT,2019-09-25 07:14:22.878000,cbb,2019-09-25 07:13:40.110000,Yellowths,1047.3,3.582581e+14,33.126274,0.982234,2019-09,9,0,-1 days +23:59:17.232000
45,uuid:6df532b2-a5e8-4ff9-b46d-0fe7f74160b7,2019-09-25 07:54:15.727,2019-10-03 09:16:23.328,0,2019-09-25 07:54:15.720,NaT,2019-09-25 07:15:13.505000,cbsd,2019-09-25 07:14:29.531000,Diseased,1047.5,3.582581e+14,33.126249,0.982233,2019-09,9,0,-1 days +23:59:16.026000
46,uuid:494a2d9c-83a7-4011-8c69-dee87f3b208b,2019-09-25 07:54:19.745,2019-10-03 09:16:25.364,0,2019-09-25 07:54:19.739,NaT,2019-09-25 07:16:09.877000,other,2019-09-25 07:15:16.400000,Rate mole,1047.8,3.582581e+14,33.126293,0.982200,2019-09,9,0,-1 days +23:59:06.523000
47,uuid:ce36921b-e4e3-4162-abca-45224849a2cd,2019-09-25 07:54:23.997,2019-10-03 09:16:26.333,0,2019-09-25 07:54:23.991,NaT,2019-09-25 07:17:07.305000,cbsd,2019-09-25 07:16:16.071000,Turned out to yellow,1048.3,3.582581e+14,33.126341,0.982176,2019-09,9,0,-1 days +23:59:08.766000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135602,uuid:87d9de62-de01-4470-937d-57035d14e9f3,2020-06-30 04:09:25.013,2020-06-30 04:09:25.013,0,2020-06-30 04:09:24.999,NaT,2020-06-26 09:56:56.269000,cmd,2020-06-26 09:54:05.501000,,1118.5,3.582461e+14,33.649752,2.043023,2020-06,6,0,-1 days +23:57:09.232000
135603,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,2020-06-30 04:09:25.807,2020-06-30 04:09:25.807,0,2020-06-30 04:09:25.801,NaT,2020-06-26 09:57:21.417000,cmd,2020-06-26 09:56:58.989000,,1118.7,3.582461e+14,33.649744,2.043025,2020-06,6,0,-1 days +23:59:37.572000
135604,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,2020-06-30 04:09:26.606,2020-06-30 04:09:26.606,0,2020-06-30 04:09:26.592,NaT,2020-06-26 09:57:47.556000,cmd,2020-06-26 09:57:23.987000,,1118.3,3.582461e+14,33.649717,2.042974,2020-06,6,0,-1 days +23:59:36.431000
135605,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,2020-06-30 04:09:27.389,2020-06-30 04:09:27.389,0,2020-06-30 04:09:27.383,NaT,2020-06-26 09:58:16.406000,cmd,2020-06-26 09:57:50.049000,,1118.5,3.582461e+14,33.649718,2.042978,2020-06,6,0,-1 days +23:59:33.643000


In [126]:
dict(list(new_cassava.groupby('IS_COMPLETE')))[1]

,ID,CREATION_DATE,LAST_UPDATE_DATE,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,CASSAVA_DATA_END_TIME,CASSAVA_IMAGE_CLASS,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,mnth_yr,month,diff,time_diff
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:37:16.450,2019-09-23 00:37:16.450,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,2019-09-23 00:36:31.097000,other,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09,9,0,-1 days +23:59:04.742000
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 13:08:04.458,2019-09-23 13:08:04.458,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,2019-09-23 12:21:17.279000,cmd,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09,9,0,-1 days +23:57:29.489000
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 13:08:21.196,2019-09-23 13:08:21.196,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,2019-09-23 12:38:01.552000,healthy,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09,9,0,-1 days +23:52:22.711000
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 13:08:26.384,2019-09-23 13:08:26.384,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,2019-09-23 12:22:28.894000,healthy,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09,9,0,-1 days +23:58:02.981000
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 13:08:37.028,2019-09-23 13:08:37.028,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,2019-09-23 12:23:32.876000,healthy,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09,9,0,-1 days +23:59:03.724000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135564,uuid:2770cbff-1119-430c-af46-4982db8418f3,2020-06-29 18:00:04.963,2020-06-29 18:00:04.963,1,2020-06-29 18:00:04.948,2020-06-29 18:00:04.948,2020-03-12 07:23:04.562000,cbsd,2020-03-12 07:21:34.575000,Yellow spotting on the leaves,1055.100000,3.582461e+14,32.896554,1.630103,2020-06,6,0,-1 days +23:58:30.013000
135565,uuid:bbf85625-8755-4310-8f56-3112a184af5d,2020-06-29 18:00:48.811,2020-06-29 18:00:48.811,1,2020-06-29 18:00:48.805,2020-06-29 18:00:48.805,2020-03-13 14:55:07.423000,cbsd,2020-03-13 14:53:34.241000,Yellow spotting on the leaves,1049.800000,3.582461e+14,32.898064,1.635019,2020-06,6,0,-1 days +23:58:26.818000
135566,uuid:9ca1c794-1053-48d3-8d4d-a51f6716e7ad,2020-06-29 18:01:44.766,2020-06-29 18:01:44.766,1,2020-06-29 18:01:44.760,2020-06-29 18:01:44.760,2020-03-13 14:56:53.479000,cbsd,2020-03-13 14:55:57.574000,Yellow stripe on the leaves,1056.400000,3.582461e+14,32.898113,1.635007,2020-06,6,0,-1 days +23:59:04.095000
135567,uuid:066c3e67-707b-4626-9b42-ffa0caeba2ce,2020-06-29 18:02:24.246,2020-06-29 18:02:24.246,1,2020-06-29 18:02:24.240,2020-06-29 18:02:24.240,2020-03-13 14:59:41.304000,cbsd,2020-03-13 14:58:18.287000,Drying of the leaves,1056.400000,3.582461e+14,32.898260,1.635032,2020-06,6,0,-1 days +23:58:36.983000


In [128]:
list(cassava['CASSAVA_GPS_LNG'].groupby(cassava['CASSAVA_GPS_LAT']).sum())

[31.7047605,
 31.7047602,
 31.7048389,
 31.704598,
 31.7045815,
 31.704836,
 31.7048151,
 31.7046501,
 31.7048382,
 31.7048314,
 31.7046508,
 31.7048025,
 31.7048096,
 31.7047633,
 31.704772,
 31.7048498,
 31.7047869,
 31.7044706,
 31.7047884,
 31.7046184,
 31.7047425,
 31.7048073,
 31.7046106,
 31.7047835,
 31.7044647,
 31.7046241,
 31.7047748,
 31.7049231,
 31.7049466,
 31.7049068,
 31.7046681,
 31.7049481,
 31.7049338,
 31.7049486,
 31.7049606,
 31.7049468,
 31.7049767,
 31.7049826,
 31.7050327,
 31.705056,
 31.7049709,
 31.7049544,
 31.7051118,
 31.7050112,
 31.7050797,
 31.7050606,
 31.7050429,
 31.705051,
 31.7051183,
 31.7051128,
 31.7050995,
 31.705058,
 31.7050794,
 31.7050382,
 31.7050928,
 31.7050343,
 31.7050935,
 31.7050372,
 31.7050433,
 31.7050417,
 31.7050018,
 31.7051282,
 31.7050201,
 31.7045284,
 31.7049842,
 31.7050388,
 31.7051233,
 31.7050396,
 31.7050359,
 31.7051283,
 31.7051244,
 63.4102066,
 31.7081374,
 31.7051135,
 31.7052385,
 31.7081904,
 31.7052455,
 31.7

In [130]:
list(cassava['CASSAVA_GPS_LNG'].groupby(cassava['CASSAVA_GPS_LAT']).mean())

[31.7047605,
 31.7047602,
 31.7048389,
 31.704598,
 31.7045815,
 31.704836,
 31.7048151,
 31.7046501,
 31.7048382,
 31.7048314,
 31.7046508,
 31.7048025,
 31.7048096,
 31.7047633,
 31.704772,
 31.7048498,
 31.7047869,
 31.7044706,
 31.7047884,
 31.7046184,
 31.7047425,
 31.7048073,
 31.7046106,
 31.7047835,
 31.7044647,
 31.7046241,
 31.7047748,
 31.7049231,
 31.7049466,
 31.7049068,
 31.7046681,
 31.7049481,
 31.7049338,
 31.7049486,
 31.7049606,
 31.7049468,
 31.7049767,
 31.7049826,
 31.7050327,
 31.705056,
 31.7049709,
 31.7049544,
 31.7051118,
 31.7050112,
 31.7050797,
 31.7050606,
 31.7050429,
 31.705051,
 31.7051183,
 31.7051128,
 31.7050995,
 31.705058,
 31.7050794,
 31.7050382,
 31.7050928,
 31.7050343,
 31.7050935,
 31.7050372,
 31.7050433,
 31.7050417,
 31.7050018,
 31.7051282,
 31.7050201,
 31.7045284,
 31.7049842,
 31.7050388,
 31.7051233,
 31.7050396,
 31.7050359,
 31.7051283,
 31.7051244,
 31.7051033,
 31.7081374,
 31.7051135,
 31.7052385,
 31.7081904,
 31.7052455,
 31.7

In [133]:
list(cassava['CASSAVA_GPS_LNG'].groupby(cassava['CASSAVA_GPS_LAT']).std())

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 1.979898989106368e-06,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.0019943239656579504,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [134]:
cassava.groupby(['CASSAVA_GPS_ACC']).mean()

,MODEL_VERSION,UI_VERSION,IS_COMPLETE,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,month
CASSAVA_GPS_ACC,,,,,,,,
2.600,0.0,0.0,1.000000,1179.400000,3.582461e+14,32.567407,0.434160,12.000000
3.000,0.0,0.0,0.831579,1106.122895,3.590707e+14,33.520212,1.327667,3.494737
3.001,0.0,0.0,1.000000,1088.000000,3.595541e+14,33.641933,1.717388,1.000000
3.004,0.0,0.0,0.000000,1106.600000,3.582581e+14,33.124734,0.982963,10.000000
3.005,0.0,0.0,0.500000,1111.200000,3.582521e+14,33.308235,0.601033,6.000000
...,...,...,...,...,...,...,...,...
9850.000,0.0,0.0,1.000000,0.000000,3.556851e+14,33.637628,1.440934,3.000000
9856.000,0.0,0.0,1.000000,0.000000,3.556851e+14,33.641421,1.438023,6.000000
9872.000,0.0,0.0,1.000000,0.000000,3.556851e+14,33.636908,1.440262,2.000000


In [135]:
cassava.groupby('CASSAVA_DATA_COMMENT').describe()

MODEL_VERSION            \
                                                           count mean  std   
CASSAVA_DATA_COMMENT                                                         
                                                         62919.0  0.0  0.0   
. Healthy looking                                            1.0  0.0  NaN   
3yrs old cassava at the garden in ainaikopi vil...           1.0  0.0  NaN   
4 months old                                                 1.0  0.0  NaN   
75 percent affected with mosaic                              1.0  0.0  NaN   
...                                                          ...  ...  ...   
narocass                                                     1.0  0.0  NaN   
severely affected stem                                       1.0  0.0  NaN   
total destruction                                            1.0  0.0  NaN   
white spot                                                   1.0  0.0  NaN   
whole field affected                                         1.0  0.0  NaN   

                                                                             \
                                                    min  25%  50%  75%  max   
CASSAVA_DATA_COMMENT                                                          
                                                    0.0  0.0  0.0  0.0  0.0   
. Healthy looking                                   0.0  0.0  0.0  0.0  0.0   
3yrs old cassava at the garden in ainaikopi vil...  0.0  0.0  0.0  0.0  0.0   
4 months old                                        0.0  0.0  0.0  0.0  0.0   
75 percent affected with mosaic                     0.0  0.0  0.0  0.0  0.0   
...                                                 ...  ...  ...  ...  ...   
narocass                                            0.0  0.0  0.0  0.0  0.0   
severely affected stem                              0.0  0.0  0.0  0.0  0.0   
total destruction                                   0.0  0.0  0.0  0.0  0.0   
white spot                                          0.0  0.0  0.0  0.0  0.0   
whole field affected                                0.0  0.0  0.0  0.0  0.0   

                                                   UI_VERSION       ...  \
                                                        count mean  ...   
CASSAVA_DATA_COMMENT                                                ...   
                                                      62919.0  0.0  ...   
. Healthy looking                                         1.0  0.0  ...   
3yrs old cassava at the garden in ainaikopi vil...        1.0  0.0  ...   
4 months old                                              1.0  0.0  ...   
75 percent affected with mosaic                           1.0  0.0  ...   
...                                                       ...  ...  ...   
narocass                                                  1.0  0.0  ...   
severely affected stem                                    1.0  0.0  ...   
total destruction                                         1.0  0.0  ...   
white spot                                                1.0  0.0  ...   
whole field affected                                      1.0  0.0  ...   

                                                   CASSAVA_GPS_ACC            \
                                                               75%       max   
CASSAVA_DATA_COMMENT                                                           
                                                            10.000  8511.000   
. Healthy looking                                            3.940     3.940   
3yrs old cassava at the garden in ainaikopi vil...           4.939     4.939   
4 months old                                                10.000    10.000   
75 percent affected with mosaic                              3.000     3.000   
...                                                            ...       ...   
narocass                                                     4.502     4.502   


## Resampling data using timeseries

In [28]:
#total amount of data resampled or added or created each month
new_cassava.resample('M', on='CREATION_DATE').CASSAVA_DATA_COMMENT.sum()


CREATION_DATE
2019-09-30    NecrosisThis plant was planted when it was sic...
2019-10-31    Looking healthy.Very serious in that area.Yell...
2019-11-30    Twisted leaflets with pale yellow leavesHealth...
2019-12-31    Green leave.Yellowing leavesYellowing folding ...
2020-01-31    Leaf chlorosisLeaf chlorosisLeaf chlorosisLeaf...
2020-02-29    CbbCbbCbbCbbCbbCbbCbbCbbCbbCbsCbbCbsCbbCbbCbbC...
2020-03-31    It's leaves are not in good condition.Some par...
2020-04-30    Yellow and green grasshopper on cassava leaves...
2020-05-31    CbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbC...
2020-06-30    Healthy growing cassava.Curled leavesBrown cur...
Freq: M, Name: CASSAVA_DATA_COMMENT, dtype: object

In [29]:
#to start from the first day of the month
new_cassava.resample('MS', on='CREATION_DATE').CASSAVA_DATA_COMMENT.sum()

CREATION_DATE
2019-09-01    NecrosisThis plant was planted when it was sic...
2019-10-01    Looking healthy.Very serious in that area.Yell...
2019-11-01    Twisted leaflets with pale yellow leavesHealth...
2019-12-01    Green leave.Yellowing leavesYellowing folding ...
2020-01-01    Leaf chlorosisLeaf chlorosisLeaf chlorosisLeaf...
2020-02-01    CbbCbbCbbCbbCbbCbbCbbCbbCbbCbsCbbCbsCbbCbbCbbC...
2020-03-01    It's leaves are not in good condition.Some par...
2020-04-01    Yellow and green grasshopper on cassava leaves...
2020-05-01    CbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbC...
2020-06-01    Healthy growing cassava.Curled leavesBrown cur...
Freq: MS, Name: CASSAVA_DATA_COMMENT, dtype: object

In [30]:
new_cassava.resample('W', on='CREATION_DATE').CASSAVA_DATA_COMMENT.sum()

CREATION_DATE
2019-09-29    NecrosisThis plant was planted when it was sic...
2019-10-06    The leaf loses the croIt has yellow colour fro...
2019-10-13    Green lineGreenBrownBrownCmdwhole field affect...
2019-10-20    Yellowish mottled leavesYellowish mottled leav...
2019-10-27    Yellowish leavesYellowish mottled leavesDeform...
2019-11-03    Signs of cbsd on stems. Nase19Signs of cbsd on...
2019-11-10    DiseasedHealthyBacterial brightHealthyGreen mi...
2019-11-17    Brown spots on leavesBrown spots on leavesBrow...
2019-11-24    CBSCbbCbbCbsCbbCbbCbbCbbCbbCbsCbbCbbCbbCmCmCmC...
2019-12-01    Brown spots and patchesGreen leaves yellowingY...
2019-12-08    Cmdnase14HealthyCombination of CBSD with CGMCo...
2019-12-15    Affected at the bottomFully affectedAffected a...
2019-12-22    Green leaf.Twisted leaf.Improved varietyInterc...
2019-12-29    Brown streakMosaicMosaicBrown streakBrown stre...
2020-01-05    CbbWhite fly is seen on the cassava leaf.Too m...
2020-01-12    Yellow and l

In [31]:
new_cassava.resample('W-MON', on='CREATION_DATE').CASSAVA_DATA_COMMENT.sum()

CREATION_DATE
2019-09-23    NecrosisThis plant was planted when it was sic...
2019-09-30    Dry sports on aleafThe leaves are yellowishThe...
2019-10-07    Looking healthy.Very serious in that area.Yell...
2019-10-14    Cassava brown spotWhitefliesWhite fliesLeave i...
2019-10-21    HealthyHealthyHealthyCbsGmGmGmGmCbbCbsGmGmCbbC...
2019-10-28    AkenaTME 14Poor crop rotation practiceNase 14L...
2019-11-04    Ariwara variety commonly grown in this areaFre...
2019-11-11    WeededAll the garden is affectedAll the garden...
2019-11-18    Cutting from affected poor sourceSome leaves a...
2019-11-25    Yellow spottedDried spotsYellow engulfed leave...
2019-12-02    Brown spots are seen on the cassava leaves.Yel...
2019-12-09    Brown streak in new varieties.Mosaic disease i...
2019-12-16    Late affectedFully affected the plantAffected ...
2019-12-23    Multiple observationDeformed leavesFoldded lea...
2019-12-30    WeedsWeeds effectsCommon weeds effectsHealthyC...
2020-01-06    Folded leafY

In [32]:
new_cassava.resample('H', on='CREATION_DATE').CASSAVA_DATA_COMMENT.sum()

CREATION_DATE
2019-09-23 00:00:00    Necrosis
2019-09-23 01:00:00           0
2019-09-23 02:00:00           0
2019-09-23 03:00:00           0
2019-09-23 04:00:00           0
                         ...   
2020-06-30 00:00:00           0
2020-06-30 01:00:00           0
2020-06-30 02:00:00           0
2020-06-30 03:00:00           0
2020-06-30 04:00:00            
Freq: H, Name: CASSAVA_DATA_COMMENT, Length: 6749, dtype: object

In [33]:
# aggregating multiple fields for each month
new_cassava.resample('M', on='CREATION_DATE').agg({'CASSAVA_DATA_COMMENT':'sum', 'CASSAVA_IMAGE_CLASS':'sum','ID':'nunique'})

,CASSAVA_DATA_COMMENT,CASSAVA_IMAGE_CLASS,ID
CREATION_DATE,,,
2019-09-30,NecrosisThis plant was planted when it was sic...,othercmdhealthyhealthyhealthycmdcbbcbbothercbs...,583
2019-10-31,Looking healthy.Very serious in that area.Yell...,healthycmdcmdcmdcbsdcmdcbsdcmdcbbcmdcmdcmdcmdc...,6668
2019-11-30,Twisted leaflets with pale yellow leavesHealth...,cmdcbsdhealthycmdcmdcmdcmdcgmcmdcmdcmdcmdcmdcm...,14428
2019-12-31,Green leave.Yellowing leavesYellowing folding ...,healthycmdcmdcbsdcmdcbbcmdcmdcmdcmdcmdcgmcmdcm...,14811
2020-01-31,Leaf chlorosisLeaf chlorosisLeaf chlorosisLeaf...,cmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdc...,24012
2020-02-29,CbbCbbCbbCbbCbbCbbCbbCbbCbbCbsCbbCbsCbbCbbCbbC...,cbbcbbcbbcbbcbbcbbcbbcbbcbbcbsdcbbcbsdcbbcbbcb...,30014
2020-03-31,It's leaves are not in good condition.Some par...,cmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdcmdc...,27653
2020-04-30,Yellow and green grasshopper on cassava leaves...,cmdcbbcbbhealthyhealthyhealthyhealthycbbcbbcbb...,2654
2020-05-31,CbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbCbbC...,cbbcbbcbbcbbcbbcbbcbbcbbcbbcbbcbbcbbcbbcbbcbbc...,4298


In [34]:
# Grouping data based on month and store type
new_cassava.groupby([pd.Grouper(key='CREATION_DATE', freq='M'), 'CASSAVA_DEVICE_ID']).CASSAVA_DATA_COMMENT.sum().head(15)

CREATION_DATE  CASSAVA_DEVICE_ID
2019-09-30     3.516321e+14                                               It's yellow
               3.582461e+14                          YellowBlack stemYellowDriedGreen
               3.582461e+14         Yellow spot on the leaf.Stardad leafPurple colour
               3.582461e+14                           Brown spots and yellowing lines
               3.582461e+14                                Yollow colourYellow colour
               3.582461e+14         The leaves are yellowishThe leaves are curledD...
               3.582461e+14               Yellow ish leavesYellowish spots forDisease
               3.582461e+14         Dry brown patches on the leafCurled and reduce...
               3.582461e+14                                                       CGM
               3.582461e+14         HealthyClean cassava tuberCBB affected plantSo...
               3.582461e+14                                    Cassava mosaic disease
               3.5824

In [35]:
#to find ocurrences of items in a dataframe
# 0 is recorded if the data image is invalid
cassava['IS_COMPLETE'].value_counts()

1    99907
0    35700
Name: IS_COMPLETE, dtype: int64

## Creating new column so as to deal with data in form of months

In [122]:
new_cassava['mnth_yr'] = new_cassava.CREATION_DATE.dt.to_period('M')
new_cassava.head(3)

,ID,CREATION_DATE,LAST_UPDATE_DATE,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,CASSAVA_DATA_END_TIME,CASSAVA_IMAGE_CLASS,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,mnth_yr,month,diff,time_diff
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:37:16.450,2019-09-23 00:37:16.450,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,2019-09-23 00:36:31.097000,other,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09,9,0,-1 days +23:59:04.742000
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 13:08:04.458,2019-09-23 13:08:04.458,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,2019-09-23 12:21:17.279000,cmd,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09,9,0,-1 days +23:57:29.489000
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 13:08:21.196,2019-09-23 13:08:21.196,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,2019-09-23 12:38:01.552000,healthy,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09,9,0,-1 days +23:52:22.711000


In [48]:
period = pd.date_range(start='2019-09-23 00:37:16.450', end='2020-06-30 04:09:28.167', freq='M')
period

DatetimeIndex(['2019-09-30 00:37:16.450000', '2019-10-31 00:37:16.450000',
               '2019-11-30 00:37:16.450000', '2019-12-31 00:37:16.450000',
               '2020-01-31 00:37:16.450000', '2020-02-29 00:37:16.450000',
               '2020-03-31 00:37:16.450000', '2020-04-30 00:37:16.450000',
               '2020-05-31 00:37:16.450000', '2020-06-30 00:37:16.450000'],
              dtype='datetime64[ns]', freq='M')

In [51]:
new_cassava['month'] = pd.DatetimeIndex(cassava['CREATION_DATE']).month
new_cassava.head()

,ID,CREATION_DATE,LAST_UPDATE_DATE,IS_COMPLETE,SUBMISSION_DATE,MARKED_AS_COMPLETE_DATE,CASSAVA_DATA_END_TIME,CASSAVA_IMAGE_CLASS,CASSAVA_DATA_START_TIME,CASSAVA_DATA_COMMENT,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,mnth_yr,month
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,2019-09-23 00:37:16.450,2019-09-23 00:37:16.450,1,2019-09-23 00:37:16.443,2019-09-23 00:37:16.443,2019-09-23 00:36:31.097000,other,2019-09-23 00:35:35.839000,Necrosis,0.000000,NaN,32.598598,0.424174,2019-09,9
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,2019-09-23 13:08:04.458,2019-09-23 13:08:04.458,1,2019-09-23 13:08:04.451,2019-09-23 13:08:04.451,2019-09-23 12:21:17.279000,cmd,2019-09-23 12:18:46.768000,This plant was planted when it was sick,0.000000,3.582461e+14,32.624897,0.521737,2019-09,9
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,2019-09-23 13:08:21.196,2019-09-23 13:08:21.196,1,2019-09-23 13:08:21.189,2019-09-23 13:08:21.189,2019-09-23 12:38:01.552000,healthy,2019-09-23 12:30:24.263000,Yellow ish leaves,1137.299927,3.582461e+14,32.625339,0.521473,2019-09,9
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,2019-09-23 13:08:26.384,2019-09-23 13:08:26.384,1,2019-09-23 13:08:26.377,2019-09-23 13:08:26.377,2019-09-23 12:22:28.894000,healthy,2019-09-23 12:20:31.875000,Healthy,1134.400000,3.582461e+14,32.625272,0.521453,2019-09,9
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,2019-09-23 13:08:37.028,2019-09-23 13:08:37.028,1,2019-09-23 13:08:37.012,2019-09-23 13:08:37.012,2019-09-23 12:23:32.876000,healthy,2019-09-23 12:22:36.600000,Clean cassava tuber,1147.500000,3.582461e+14,32.625380,0.521361,2019-09,9


In [53]:
cassava['month_year'] = pd.to_datetime(cassava['CREATION_DATE']).dt.to_period('M')
cassava.iloc[:,[0,22,23]]

,ID,month,month_year
0,uuid:793b08fd-4d24-42a0-835c-fede2e71ec14,9,2019-09
1,uuid:e235fa16-536d-4e2b-a463-95d32407f852,9,2019-09
2,uuid:4806eecb-f8a6-46ae-b34c-58a5e02e8cee,9,2019-09
3,uuid:0ff8ad92-8d31-45e9-8b16-7010939e5f96,9,2019-09
4,uuid:47ed731b-98e7-426e-a2f0-138b1e0ddb48,9,2019-09
...,...,...,...
135602,uuid:87d9de62-de01-4470-937d-57035d14e9f3,6,2020-06
135603,uuid:94ffe6af-0d96-4305-bb08-6a1255777f8e,6,2020-06
135604,uuid:5b6dac9b-ad53-46d7-989b-9cc21422650c,6,2020-06
135605,uuid:fd0c81d0-c785-4151-a034-e7ac6885a65c,6,2020-06


In [54]:
cassava.groupby('month_year').sum()

,MODEL_VERSION,UI_VERSION,IS_COMPLETE,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,CASSAVA_GPS_ACC,month
month_year,,,,,,,,,
2019-09,0.0,0.0,364,6.382039e+05,2.027644e+17,1.940336e+04,650.214049,2.571889e+04,5247
2019-10,0.0,0.0,3046,7.319219e+06,2.382336e+18,2.213052e+05,7356.303769,2.565582e+05,66680
2019-11,0.0,0.0,9542,1.396046e+07,5.163183e+18,4.801345e+05,18815.079750,7.395864e+06,158708
2019-12,0.0,0.0,10029,1.593762e+07,5.305195e+18,4.898953e+05,20556.609721,1.259749e+06,177732
2020-01,0.0,0.0,18576,2.589100e+07,8.589938e+18,7.951711e+05,37839.791322,1.324724e+06,24012
2020-02,0.0,0.0,24956,2.508036e+07,1.073438e+19,1.000067e+06,39561.993714,3.274966e+07,60028
2020-03,0.0,0.0,21020,2.539524e+07,9.895403e+18,9.185614e+05,34516.399057,2.010406e+07,82959
2020-04,0.0,0.0,2641,2.896839e+06,9.508049e+17,8.776440e+04,3009.204882,3.986300e+04,10616
2020-05,0.0,0.0,1884,4.741756e+06,1.539413e+18,1.427513e+05,5007.150462,3.352646e+04,21490


In [55]:
new_cassava.columns

Index(['ID', 'CREATION_DATE', 'LAST_UPDATE_DATE', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'mnth_yr', 'month'],
      dtype='object')

In [56]:
new_cassava.columns

Index(['ID', 'CREATION_DATE', 'LAST_UPDATE_DATE', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'mnth_yr', 'month'],
      dtype='object')

# Fitering using a boolean

In [63]:
#To filter the dataset to get a smaller dataframe with "month" values equals to September
is_9 = new_cassava['month'] == 9
print(is_9)

0          True
1          True
2          True
3          True
4          True
          ...  
135602    False
135603    False
135604    False
135605    False
135606    False
Name: month, Length: 135607, dtype: bool


In [64]:
#Using the boolean variable to filter the dataframe. We shall see that after subsetting, we get a much smaller dataframe
new_cassava_9 = new_cassava[is_9]
print(new_cassava_9.shape)

(583, 16)


In [68]:
is_10 = new_cassava['month'] == 10
#print(is_10)
new_cassava_10 = new_cassava[is_10]
print(new_cassava_10.shape)

(6668, 16)


In [69]:
is_11 = new_cassava['month'] == 11
#print(is_11)
new_cassava_11 = new_cassava[is_11]
print(new_cassava_11.shape)

(14428, 16)


In [70]:
is_12 = new_cassava['month'] == 12
#print(is_12)
new_cassava_12 = new_cassava[is_12]
print(new_cassava_12.shape)

(14811, 16)


In [71]:
is_1 = new_cassava['month'] == 1
#print(is_1)
new_cassava_1 = new_cassava[is_1]
print(new_cassava_1.shape)

(24012, 16)


In [72]:
is_2 = new_cassava['month'] == 2
#print(is_2)
new_cassava_2 = new_cassava[is_2]
print(new_cassava_2.shape)

(30014, 16)


In [73]:
is_3 = new_cassava['month'] == 3
#print(is_3)
new_cassava_3 = new_cassava[is_3]
print(new_cassava_3.shape)

(27653, 16)


In [75]:
is_4 = new_cassava['month'] == 4
#print(is_4)
new_cassava_4 = new_cassava[is_4]
print(new_cassava_4.shape)

(2654, 16)


In [76]:
is_5 = new_cassava['month'] == 5
#print(is_5)
new_cassava_5 = new_cassava[is_5]
print(new_cassava_5.shape)

(4298, 16)


In [77]:
is_6 = new_cassava['month'] == 6
#print(is_6)
new_cassava_6 = new_cassava[is_6]
print(new_cassava_6.shape)

(10486, 16)


In [79]:
is_7 = new_cassava['month'] == 7
#print(is_7)
new_cassava_7 = new_cassava[is_7]
print(new_cassava_7.shape)

(0, 16)


In [80]:
new_cassava.shape

(135607, 16)

In [88]:
#proof of concept
#Is the number of rows in each month equal to the total number of columns in entire dataset? 
sum = 10486+4298+2654+27653+30014+24012+14811+14428+6668+583
is_sum = sum == 135607
print(is_sum)
        
        
    

True


In [90]:
new_cassava.columns

Index(['ID', 'CREATION_DATE', 'LAST_UPDATE_DATE', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'mnth_yr', 'month'],
      dtype='object')

# To find out how long users too to fill in the data

In [101]:
#Use the start time and end time
#First group by start time and end time
new_cassava.groupby('CASSAVA_DATA_START_TIME').sum()

,IS_COMPLETE,CASSAVA_GPS_ALT,CASSAVA_DEVICE_ID,CASSAVA_GPS_LNG,CASSAVA_GPS_LAT,month
CASSAVA_DATA_START_TIME,,,,,,
2017-01-01 00:01:24.138000,1,1153.0,3.582461e+14,32.534426,0.723380,12
2017-01-01 00:01:34.534000,1,1128.9,3.582461e+14,33.631792,1.795903,3
2017-01-01 00:01:56.777000,1,1086.3,3.582461e+14,32.878515,2.276636,1
2017-01-01 00:02:03.748000,1,1046.4,3.582461e+14,32.663748,1.543455,1
2017-01-01 00:02:19.416000,0,1075.3,3.582461e+14,33.069219,1.741650,6
...,...,...,...,...,...,...
2020-06-29 06:27:05.615000,1,1167.0,3.582461e+14,30.174335,0.331975,6
2020-06-29 06:38:21.965000,1,1180.6,3.582461e+14,30.173798,0.333103,6
2020-06-29 07:50:18.078000,0,1264.5,3.582461e+14,32.186240,0.910024,6


In [111]:
new_cassava.columns

Index(['ID', 'CREATION_DATE', 'LAST_UPDATE_DATE', 'IS_COMPLETE',
       'SUBMISSION_DATE', 'MARKED_AS_COMPLETE_DATE', 'CASSAVA_DATA_END_TIME',
       'CASSAVA_IMAGE_CLASS', 'CASSAVA_DATA_START_TIME',
       'CASSAVA_DATA_COMMENT', 'CASSAVA_GPS_ALT', 'CASSAVA_DEVICE_ID',
       'CASSAVA_GPS_LNG', 'CASSAVA_GPS_LAT', 'mnth_yr', 'month', 'diff'],
      dtype='object')

In [114]:
#Looking at the result, it shows that all data was submitted on the same day it was created.
new_cassava['diff'] = (new_cassava['CREATION_DATE'] - new_cassava['SUBMISSION_DATE']).dt.days
new_cassava.iloc[:,[1,4,16]]

,CREATION_DATE,SUBMISSION_DATE,diff
0,2019-09-23 00:37:16.450,2019-09-23 00:37:16.443,0
1,2019-09-23 13:08:04.458,2019-09-23 13:08:04.451,0
2,2019-09-23 13:08:21.196,2019-09-23 13:08:21.189,0
3,2019-09-23 13:08:26.384,2019-09-23 13:08:26.377,0
4,2019-09-23 13:08:37.028,2019-09-23 13:08:37.012,0
...,...,...,...
135602,2020-06-30 04:09:25.013,2020-06-30 04:09:24.999,0
135603,2020-06-30 04:09:25.807,2020-06-30 04:09:25.801,0
135604,2020-06-30 04:09:26.606,2020-06-30 04:09:26.592,0
135605,2020-06-30 04:09:27.389,2020-06-30 04:09:27.383,0


In [118]:
new_cassava['time_diff'] = pd.to_datetime(new_cassava.CASSAVA_DATA_START_TIME) - pd.to_datetime(new_cassava.CASSAVA_DATA_END_TIME)
new_cassava.iloc[:,[8,6,17]]

,CASSAVA_DATA_START_TIME,CASSAVA_DATA_END_TIME,time_diff
0,2019-09-23 00:35:35.839000,2019-09-23 00:36:31.097000,-1 days +23:59:04.742000
1,2019-09-23 12:18:46.768000,2019-09-23 12:21:17.279000,-1 days +23:57:29.489000
2,2019-09-23 12:30:24.263000,2019-09-23 12:38:01.552000,-1 days +23:52:22.711000
3,2019-09-23 12:20:31.875000,2019-09-23 12:22:28.894000,-1 days +23:58:02.981000
4,2019-09-23 12:22:36.600000,2019-09-23 12:23:32.876000,-1 days +23:59:03.724000
...,...,...,...
135602,2020-06-26 09:54:05.501000,2020-06-26 09:56:56.269000,-1 days +23:57:09.232000
135603,2020-06-26 09:56:58.989000,2020-06-26 09:57:21.417000,-1 days +23:59:37.572000
135604,2020-06-26 09:57:23.987000,2020-06-26 09:57:47.556000,-1 days +23:59:36.431000
135605,2020-06-26 09:57:50.049000,2020-06-26 09:58:16.406000,-1 days +23:59:33.643000


# SUMMARY STATISTICS


In [91]:
#Therefore this is the number of times data was collected in each month from September of 2019 to June of 2020
new_cassava['month'].value_counts()

2     30014
3     27653
1     24012
12    14811
11    14428
6     10486
10     6668
5      4298
4      2654
9       583
Name: month, dtype: int64

In [97]:
#summary statistics for image classes shows how many crops were suffering from a certain disease.
new_cassava['CASSAVA_IMAGE_CLASS'].value_counts()

cmd        77282
cbb        23088
cbsd       15491
healthy     8725
cgm         7981
other       3040
Name: CASSAVA_IMAGE_CLASS, dtype: int64

In [98]:
#number of valid and invalid images captured by the users
new_cassava['IS_COMPLETE'].value_counts()

1    99907
0    35700
Name: IS_COMPLETE, dtype: int64

In [116]:
#number of times each comment appeared
new_cassava['CASSAVA_DATA_COMMENT'].value_counts()

                                                                                                        62919
Cbb                                                                                                     16791
Pale yellow leave                                                                                       13872
Paleyellow leave                                                                                         5191
Cm                                                                                                       4181
                                                                                                        ...  
Leaves narrowing in ongijii village, opuure parish, atiira subcounty, serere county, serere district        1
Yellowing folding leaves with some drying patches on leaves                                                 1
multiple observation                                                                                        1
Looks good

# Dealing with image data

In [ ]:
cassava.iloc[:,[0,7,11,21]]

In [ ]:
new_cassava_valid = new_cassava[new_cassava['IS_COMPLETE'] == 1]
new_cassava_valid

In [ ]:
new_cassava_invalid = new_cassava[new_cassava['IS_COMPLETE'] == 0]
new_cassava_invalid